In [12]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [13]:
##Load the trained model, scaler, and encoder
model = load_model('churn_model.h5')

#load the scaler and encoder
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('label_enoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open('onehotencoder_geography.pkl', 'rb') as f:
    onehotencoder = pickle.load(f)



In [14]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender':'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [15]:
# One-hot encode 'Geography'
geo_encoded = onehotencoder.transform([[input_data['Geography']]])
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehotencoder.get_feature_names_out(['Geography']))
geo_encoded_df


/Applications/anaconda3/envs/ann_env/lib/python3.11/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [16]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [18]:
##Encoding categorical var
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [19]:
# Concatenate one-hot encoded geography columns
input_df = pd.concat([input_df, geo_encoded_df], axis=1)
input_df.drop('Geography', axis=1, inplace=True)

In [20]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [21]:
##Scaling the input data
scaled_input = scaler.transform(input_df)
scaled_input

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [22]:
##Predict churn
prediction = model.predict(scaled_input)
prediction

2025-12-30 19:39:54.063667: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step


array([[0.05906698]], dtype=float32)

In [23]:
prediction_probability = prediction[0][0]
prediction_probability

0.059066977

In [24]:
if prediction_probability > 0.5:
    print(f"The customer is likely to churn with a probability of {prediction_probability:.2f}")
else:
    print(f"The customer is unlikely to churn with a probability of {prediction_probability:.2f}")

The customer is unlikely to churn with a probability of 0.06
